In [19]:
import tator

from tator.util import clone_localization_list

In [3]:
HOST = "https://tator.whoi.edu"
TOKEN = open("tator_token.txt", "r").readlines()[0].strip()

FISH_DETECTION_PROJECT = 2
ANIMAL_BBOX_LOCALIZTION_TYPE = 2

In [4]:
api = tator.get_api(HOST, TOKEN)

In [5]:
api.get_project_list()

[{'backup_bucket': None,
  'bucket': None,
  'created': '2023-02-21T20:13:28.968Z',
  'duration': 0,
  'enable_downloads': True,
  'id': 2,
  'name': 'Fish Detection',
  'num_files': 0,
  'organization': 2,
  'permission': 'Full Control',
  'size': 0,
  'summary': '',
  'thumb': None,
  'upload_bucket': None,
  'usernames': []}]

## Getting / deleting localization lists

In [ ]:
media_ids = [437, 438]

localization_list = api.get_localization_list(
    project=FISH_DETECTION_PROJECT, 
    type=ANIMAL_BBOX_LOCALIZTION_TYPE,
    media_id=media_ids
)
print(len(localization_list))


In [ ]:
media_ids = [437, 438]

ret = api.delete_localization_list(
    project=FISH_DETECTION_PROJECT, 
    type=ANIMAL_BBOX_LOCALIZTION_TYPE,
    media_id=media_ids
)
print(ret)

## Deleting versions (layers)

In [6]:
api.get_version_list(project=FISH_DETECTION_PROJECT)

[{'bases': [],
  'created_by': None,
  'description': 'Initial version',
  'id': 2,
  'name': 'Baseline',
  'number': 0,
  'project': 2,
  'show_empty': True},
 {'bases': [],
  'created_by': None,
  'description': '',
  'id': 4,
  'name': 'Manual First Pass',
  'number': 1,
  'project': 2,
  'show_empty': True},
 {'bases': [],
  'created_by': None,
  'description': '',
  'id': 5,
  'name': 'Baseline_manual',
  'number': 2,
  'project': 2,
  'show_empty': True}]

In [17]:
version_id = 4
print(api.get_version(id=version_id))
res = input(f"Are you sure you want to delete version {version_id}? (y/n")
if res == 'y':
    print(f"deleting...")
    api.delete_version(id=version_id)
else:
    print(f"no action taken")

{'bases': [],
 'created_by': None,
 'description': '',
 'id': 4,
 'name': 'Manual First Pass',
 'number': 1,
 'project': 2,
 'show_empty': True}


Are you sure you want to delete version 4? (y/n 


no action taken


## Cloning layers

In [21]:
query_params = {'project': 2, 'media_id': [437, 438]}
dest_project = 2
version_mapping = {2: 5}
media_mapping = {437: 437, 438: 438}
localization_type_mapping = {2: 2}
created_ids = []
generator = clone_localization_list(api, query_params, dest_project, version_mapping,
                                    media_mapping, localization_type_mapping)
for num_created, num_total, response, id_map in generator:
    print(f"Created {num_created} of {num_total} localizations...")
    created_ids += response.id
print(f"Finished creating {num_created} localizations!")

Created 500 of 170712 localizations...
Created 1000 of 170712 localizations...
Created 1500 of 170712 localizations...
Created 2000 of 170712 localizations...
Created 2500 of 170712 localizations...
Created 3000 of 170712 localizations...
Created 3500 of 170712 localizations...
Created 4000 of 170712 localizations...
Created 4500 of 170712 localizations...
Created 5000 of 170712 localizations...
Created 5500 of 170712 localizations...
Created 6000 of 170712 localizations...
Created 6500 of 170712 localizations...
Created 7000 of 170712 localizations...
Created 7500 of 170712 localizations...
Created 8000 of 170712 localizations...
Created 8500 of 170712 localizations...
Created 9000 of 170712 localizations...
Created 9500 of 170712 localizations...
Created 10000 of 170712 localizations...
Created 10500 of 170712 localizations...
Created 11000 of 170712 localizations...
Created 11500 of 170712 localizations...
Created 12000 of 170712 localizations...
Created 12500 of 170712 localization